<a href="https://colab.research.google.com/github/SheKJana/first-own-repository/blob/main/Nach_KAQs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Semesteraufgabe BINA- nach KAQs gegliedert**

In [9]:
#Vorbereitung Blibliotheken und Visualisierungstools hochalde
!pip install scikit-learn
!pip install kaggle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Markdown


In [17]:
#NHANES (aus Kaggel Quelle) hochladen und zusammenführen
import pandas as pd

diet = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/diet.csv")
examination = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/examination.csv")
questionnaire = pd.read_csv("https://raw.githubusercontent.com/iceangel79/MScWinf_Gruppenarbeit_Sem2/refs/heads/main/Semesterarbeit/Datasets/Kaggle/questionnaire.csv")

# 2. Relevante Spalten auswählen
diet_cols = [
    'SEQN', 'DR1TKCAL', 'DR1TSUGR', 'DR1TTFAT', 'DR1TSODI',
    'DR1TCALC', 'DR1TCARB', 'DR1TCHOL', 'DR1TIRON', 'DR1TPHOS',
    'DR1TMFAT', 'DR1TPFAT', 'DR1TSFAT', 'DR1TPROT', 'DR1TZINC'
]

examination_cols = ['SEQN', 'BMXBMI', 'BPXSY1', 'BPXDI1']

questionnaire_cols = [
    'SEQN', 'DIQ010',  # Diabetes
    'BPQ020', 'BPQ030', 'BPQ050A', 'BPQ080', 'BPQ090D',  # Blutdruck & Cholesterin
    'MCQ080', 'MCQ220',  # Übergewicht, Krebs
    'PAD615', 'DID040', 'DIQ160', 'DIQ170',  # Aktivität, Diabetes-Details
    'WHD010', 'WHD020',  # Selbstberichtete Größe/Gewicht
    'DED120', 'DED125',  # Sonneneinstrahlung
    'MCQ086',  # Gluten
    'HIQ011', 'HUQ010',  # Versicherung, allgemeine Gesundheit
    'SMD460', 'SMAQUEX2', 'SMDANY', 'SMQ710', 'SMQ621',  # Rauchen
    'IND235', 'INDFMMPC',  # Einkommen, Armutslevel
    'CBQ596', 'CBQ611',  # MyPlate bekannt/verwendet
    'DBD910', 'CBD070',  # Frozen Meals, Ausgaben für Essen
    'ALQ110', 'ALQ120Q', 'ALQ151',  # Alkohol
    'FSD032C', 'FSD151'  # Nahrungsunsicherheit
]

# 3. Relevante Daten extrahieren
diet_small = diet[[col for col in diet_cols if col in diet.columns]]
examination_small = examination[[col for col in examination_cols if col in examination.columns]]
questionnaire_small = questionnaire[[col for col in questionnaire_cols if col in questionnaire.columns]]

# 4. Zusammenführen
df = diet_small.merge(examination_small, on="SEQN", how="inner")
df = df.merge(questionnaire_small, on="SEQN", how="inner")


In [21]:
#Bereinigen der Daten

# Liste der zwingend erforderlichen Spalten
pflicht_spalten = [
    'DIQ010',      # Arzt diagnostizierte Diabetes
    'BMXBMI',      # BMI
    'BPXSY1',      # Blutdruck (systolisch)
    'BPXDI1',      # Blutdruck (diastolisch)
    'DR1TKCAL',    # Kalorien
    'DR1TSUGR',    # Zucker
    'DR1TTFAT',    # Fett
    'DR1TSODI',    # Natrium
    'RIDAGEYR',    # Alter
    'RIAGENDR'     # Geschlecht
]

# Erkennung und Entfernung von Zeilen mit fehlenden Pflichtwerten
df_filtered = df.dropna(subset=pflicht_spalten).copy()

# Optional: Kennzeichnung, wie viele optionale Werte fehlen (für spätere Qualitätsanalysen)
df_filtered['anzahl_missing_felder'] = df_filtered.isna().sum(axis=1)


KeyError: ['RIDAGEYR', 'RIAGENDR']

In [19]:
#Feature erstellen für Krankheiten - Übergewicht; Bluthochdruck und Diabetes
# . Gesundheitsstatus klassifizieren
def classify_health(row):
    if row['DIQ010'] == 1:
        return 'Diabetes'
    elif row['BPXSY1'] >= 130 or row['BPXDI1'] >= 80:
        return 'Bluthochdruck'
    elif row['BMXBMI'] >= 25:
        return 'Übergewicht'
    else:
        return 'Gesund'

df_clean.loc[:, 'Gesundheitsstatus'] = df_clean.apply(classify_health, axis=1)

# Exportieren & Vorschau
df_clean.to_csv("NHANES_bereinigt.csv", index=False)
print(df_clean[['SEQN', 'Gesundheitsstatus'] + diet_cols[1:] + examination_cols[1:]].head())


Empty DataFrame
Columns: [SEQN, Gesundheitsstatus, DR1TKCAL, DR1TSUGR, DR1TTFAT, DR1TSODI, DR1TCALC, DR1TCARB, DR1TCHOL, DR1TIRON, DR1TPHOS, DR1TMFAT, DR1TPFAT, DR1TSFAT, DR1TPROT, DR1TZINC, BMXBMI, BPXSY1, BPXDI1]
Index: []
Empty DataFrame
Columns: [SEQN, Gesundheitsstatus, DR1TKCAL, DR1TSUGR, DR1TTFAT, DR1TSODI, DR1TCALC, DR1TCARB, DR1TCHOL, DR1TIRON, DR1TPHOS, DR1TMFAT, DR1TPFAT, DR1TSFAT, DR1TPROT, DR1TZINC, BMXBMI, BPXSY1, BPXDI1]
Index: []


In [20]:
#Feature engeenering für Klassifikation von: "" nicht raucher, passiv Raucher, gelegenheits Raucher, regelmässiger Raucher"

def klassifiziere_raucher(row):
    smq621 = row.get("SMQ621")
    smq710 = row.get("SMQ710")
    smd460 = row.get("SMD460")

    if pd.isna(smq621) and pd.isna(smq710) and pd.isna(smd460):
        return "keine_angaben"

    if not pd.isna(smq710) and smq710 >= 3:
        return "regelmässiger_raucher"
    elif not pd.isna(smq710) and 1 <= smq710 <= 2:
        return "gelegentlicher_raucher"
    elif not pd.isna(smq621) and smq621 >= 100:
        return "regelmässiger_raucher"
    elif not pd.isna(smq621) and 1 <= smq621 < 100:
        return "gelegentlicher_raucher"
    elif not pd.isna(smd460) and smd460 >= 1:
        return "passivraucher"
    else:
        return "nichtraucher"

def klassifiziere_alkohol(row):
    alq110 = row.get("ALQ110")
    alq120q = row.get("ALQ120Q")

    if pd.isna(alq110) and pd.isna(alq120q):
        return "keine_angaben"

    if alq110 == 2:
        return "konsumiert_keinen_alkohol"

    if not pd.isna(alq120q):
        if alq120q == 0:
            return "konsumiert_keinen_alkohol"
        elif 1 <= alq120q <= 3:
            return "gelegentlich"
        elif 4 <= alq120q <= 6:
            return "regelmässig"
        elif alq120q >= 7:
            return "regelmässig_und_viel"

    return "keine_angaben"

# Neue Spalten erstellen
questionnaire["rauchverhalten_kategorie"] = questionnaire.apply(klassifiziere_raucher, axis=1)
questionnaire["alkoholkonsum_kategorie"] = questionnaire.apply(klassifiziere_alkohol, axis=1)

# Vorschau (optional)
questionnaire[["SEQN", "rauchverhalten_kategorie", "alkoholkonsum_kategorie"]].head()


,SEQN,rauchverhalten_kategorie,alkoholkonsum_kategorie
0,73557,passivraucher,gelegentlich
1,73558,gelegentlicher_raucher,regelmässig_und_viel
2,73559,nichtraucher,konsumiert_keinen_alkohol
3,73560,passivraucher,keine_angaben
4,73561,nichtraucher,konsumiert_keinen_alkohol
